In [392]:
from randomuser import RandomUser
import pandas as pd
import requests
import random
from IPython.utils import io
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from pathlib import Path
from dotenv import load_dotenv
import os
from googleapiclient.http import MediaFileUpload
import gspread
import json


In [393]:
"""
Tarefa 1

Usar a API do Random User Generator para gerar uma lista de 1.000 usuários brasileiros
"""

users = RandomUser.generate_users(1000, {"nat": "br"})

print(len(users))

1000


In [394]:
"""
Tarefa 1.1

Faça uma tabela com todas as informações desses usuários
"""

users_info = []
for user in users:
    user_info = {
        "first_name": user.get_first_name(),
        "last_name": user.get_last_name(),
        "gender": user.get_gender(),
        "email": user.get_email(),
        "cell": user.get_cell(),
        "age": user.get_age(),
    }
    users_info.append(user_info)

users_df = pd.DataFrame(users_info)

print(users_df.head().to_markdown())

|    | first_name   | last_name   | gender   | email                           | cell           |   age |
|---:|:-------------|:------------|:---------|:--------------------------------|:---------------|------:|
|  0 | Guimar       | da Mata     | male     | guimar.damata@example.com       | (29) 5636-9110 |    67 |
|  1 | Carlúcia     | Vieira      | female   | carlucia.vieira@example.com     | (59) 9856-0868 |    30 |
|  2 | Clodomiro    | Viana       | male     | clodomiro.viana@example.com     | (47) 8176-9128 |    59 |
|  3 | Evangelista  | Silva       | male     | evangelista.silva@example.com   | (41) 8319-1577 |    70 |
|  4 | Fulgêncio    | Fernandes   | male     | fulgencio.fernandes@example.com | (66) 2726-3136 |    29 |


In [395]:
"""
Tarefa 1.2

Crie uma coluna que indique o Estado do usuário, baseado em seu telefone celular
Se o telefone celular tiver um DDD inexistente remova o usuário da tabela
"""


def get_state_from_ddd(ddd):
    state = {
        "11": "São Paulo",
        "12": "São Paulo",
        "13": "São Paulo",
        "14": "São Paulo",
        "15": "São Paulo",
        "16": "São Paulo",
        "17": "São Paulo",
        "18": "São Paulo",
        "19": "São Paulo",
        "21": "Rio de Janeiro",
        "22": "Rio de Janeiro",
        "24": "Rio de Janeiro",
        "27": "Espírito Santo",
        "28": "Espírito Santo",
        "31": "Minas Gerais",
        "32": "Minas Gerais",
        "33": "Minas Gerais",
        "34": "Minas Gerais",
        "35": "Minas Gerais",
        "37": "Minas Gerais",
        "38": "Minas Gerais",
        "41": "Paraná",
        "42": "Paraná",
        "43": "Paraná",
        "44": "Paraná",
        "45": "Paraná",
        "46": "Paraná",
        "47": "Santa Catarina",
        "48": "Santa Catarina",
        "49": "Santa Catarina",
        "51": "Rio Grande do Sul",
        "53": "Rio Grande do Sul",
        "54": "Rio Grande do Sul",
        "55": "Rio Grande do Sul",
        "61": "Distrito Federal",
        "62": "Goiás",
        "63": "Tocantins",
        "64": "Goiás",
        "65": "Mato Grosso",
        "66": "Mato Grosso",
        "67": "Mato Grosso do Sul",
        "68": "Acre",
        "69": "Rondônia",
        "71": "Bahia",
        "73": "Bahia",
        "74": "Bahia",
        "75": "Bahia",
        "77": "Bahia",
        "79": "Sergipe",
        "81": "Pernambuco",
        "82": "Alagoas",
        "83": "Paraíba",
        "84": "Rio Grande do Norte",
        "85": "Ceará",
        "86": "Piauí",
        "87": "Pernambuco",
        "88": "Ceará",
        "89": "Piauí",
        "91": "Pará",
        "92": "Amazonas",
        "93": "Pará",
        "94": "Pará",
        "95": "Roraima",
        "96": "Amapá",
        "97": "Amazonas",
        "98": "Maranhão",
        "99": "Maranhão",
    }
    return state.get(ddd, None)


for index, user_row in users_df.iterrows():
    ddd = user_row.cell[1:3]
    state = get_state_from_ddd(ddd)

    if state:
        users_df.loc[index, "state"] = state
    else:
        users_df.drop(index, inplace=True)

users_df.reset_index(drop=True, inplace=True)

print(users_df.head().to_markdown())

|    | first_name   | last_name   | gender   | email                           | cell           |   age | state          |
|---:|:-------------|:------------|:---------|:--------------------------------|:---------------|------:|:---------------|
|  0 | Clodomiro    | Viana       | male     | clodomiro.viana@example.com     | (47) 8176-9128 |    59 | Santa Catarina |
|  1 | Evangelista  | Silva       | male     | evangelista.silva@example.com   | (41) 8319-1577 |    70 | Paraná         |
|  2 | Fulgêncio    | Fernandes   | male     | fulgencio.fernandes@example.com | (66) 2726-3136 |    29 | Mato Grosso    |
|  3 | Balbino      | Almeida     | male     | balbino.almeida@example.com     | (18) 2615-8313 |    70 | São Paulo      |
|  4 | Odara        | Silveira    | female   | odara.silveira@example.com      | (44) 5816-3343 |    38 | Paraná         |


In [396]:
"""
Tarefa 2

Baixe os Pokémons iniciais de todas as gerações por meio da PokeAPI
"""

initial_pokemons = [
    "bulbasaur",
    "charmander",
    "squirtle",
    "pikachu",
    "chikorita",
    "cyndaquil",
    "totodile",
    "treecko",
    "torchic",
    "mudkip",
    "turtwig",
    "chimchar",
    "piplup",
    "snivy",
    "tepig",
    "oshawott",
    "chespin",
    "fennekin",
    "froakie",
    "rowlet",
    "litten",
    "popplio",
    "grookey",
    "scorbunny",
    "sobble",
]

url = "https://pokeapi.co/api/v2/pokemon/"

pokemons = []
for pokemon_name in initial_pokemons:
    url_pokemon = url + f"{pokemon_name}"
    response = requests.get(url_pokemon)
    if response.status_code == 200:
        pokemons.append(response.json())
        print(f"{pokemon_name} - {response}")
    else:
        print(f"{pokemon_name} - {response}")

bulbasaur - <Response [200]>
charmander - <Response [200]>
squirtle - <Response [200]>
pikachu - <Response [200]>
chikorita - <Response [200]>
cyndaquil - <Response [200]>
totodile - <Response [200]>
treecko - <Response [200]>
torchic - <Response [200]>
mudkip - <Response [200]>
turtwig - <Response [200]>
chimchar - <Response [200]>
piplup - <Response [200]>
snivy - <Response [200]>
tepig - <Response [200]>
oshawott - <Response [200]>
chespin - <Response [200]>
fennekin - <Response [200]>
froakie - <Response [200]>
rowlet - <Response [200]>
litten - <Response [200]>
popplio - <Response [200]>
grookey - <Response [200]>
scorbunny - <Response [200]>
sobble - <Response [200]>


In [397]:
"""
Tarefa 2.1

Atribua um Pokémon inicial aleatório para cada um dos usuários
"""

for user_index, user_row in users_df.iterrows():
    users_df.loc[user_index, "pokemon"] = random.choice(initial_pokemons)

print(users_df.head().to_markdown())

|    | first_name   | last_name   | gender   | email                           | cell           |   age | state          | pokemon   |
|---:|:-------------|:------------|:---------|:--------------------------------|:---------------|------:|:---------------|:----------|
|  0 | Clodomiro    | Viana       | male     | clodomiro.viana@example.com     | (47) 8176-9128 |    59 | Santa Catarina | popplio   |
|  1 | Evangelista  | Silva       | male     | evangelista.silva@example.com   | (41) 8319-1577 |    70 | Paraná         | torchic   |
|  2 | Fulgêncio    | Fernandes   | male     | fulgencio.fernandes@example.com | (66) 2726-3136 |    29 | Mato Grosso    | bulbasaur |
|  3 | Balbino      | Almeida     | male     | balbino.almeida@example.com     | (18) 2615-8313 |    70 | São Paulo      | chikorita |
|  4 | Odara        | Silveira    | female   | odara.silveira@example.com      | (44) 5816-3343 |    38 | Paraná         | mudkip    |


In [398]:
"""
Tarefa 3.a

Conecte com o Google Drive
Disponível em: https://drive.google.com/drive/folders/1GvSdaQwIXeNE1KBhb-3Z-r-Sp_fCT2-E?usp=sharing
"""

SCOPES = ["https://www.googleapis.com/auth/drive"]
credentials_json = Path("..") / "credentials.json"
flow = InstalledAppFlow.from_client_secrets_file(credentials_json, SCOPES)
with io.capture_output() as captured:
    credentials = flow.run_local_server(port=0)
service = build("drive", "v3", credentials=credentials)

In [399]:
"""
Tarefa 3.b

Crie uma pasta para cada elemento dos Pokémons
"""

pokemon_types = []
for pokemon in pokemons:
    for type_item in pokemon["types"]:
        pokemon_types.append(type_item["type"]["name"])
pokemon_types = list(set(pokemon_types))


env_path = Path("..") / ".env"
load_dotenv(dotenv_path=env_path)

greenpeace_folder_id = os.environ.get("GOOGLE_DRIVE_GREENPEACE_FOLDER_ID")
try:
    files = service.files().list(q=f"'{greenpeace_folder_id}' in parents").execute()
except HttpError as err:
    print(err)

file_names_list = []
for file in files["files"]:
    file_names_list.append(file["name"])

google_drive_folder_id_by_type = {}
for pokemon_type in pokemon_types:
    if pokemon_type not in file_names_list:
        try:
            file_metadata = {
                "name": pokemon_type,
                "mimeType": "application/vnd.google-apps.folder",
                "parents": [greenpeace_folder_id],
            }
            file = service.files().create(body=file_metadata).execute()
            google_drive_folder_id_by_type[pokemon_type] = file.get("id")
            print(f"Folder for type *{pokemon_type}* created successfully with ID: {file.get('id')}")

        except HttpError as err:
            print(f"{err} for type *{pokemon_type}*")
    else:
        print(f"Folder for type *{pokemon_type}* already exists")

Folder for type *grass* created successfully with ID: 1i_h8iSixiEnoXgkHUCGn_gfX4Yyxa1Gw
Folder for type *electric* created successfully with ID: 1zptJq5hbPJweNDhUT3cPY_lo-aKlRc_J
Folder for type *flying* created successfully with ID: 11cmXfgoAT2oLiUp15QcBi-krz2z87RGP
Folder for type *fire* created successfully with ID: 1-35HN45ksh0ldfEqRejIEdxhX2x04wTe
Folder for type *water* created successfully with ID: 16dhjTNwQvau3M257txU6dg2n3GEXNXoS
Folder for type *poison* created successfully with ID: 1wKx3eKli6IWSOQXMga8x5e2z-BaV0r5u


In [400]:
"""
Tarefa 3.1

Adicione em cada uma das pastas dois arquivos em CSV
Cada pasta deve ter dois arquivos um com todos os homens com Pokémons
daquele tipo, e outro com todas as mulheres com Pokémons daquele tipo
"""

pokemon_by_type = {pokemon_type: [] for pokemon_type in pokemon_types}

for pokemon in pokemons:
    for type_item in pokemon["types"]:
        for pokemon_type in pokemon_types:
            if type_item["type"]["name"] == pokemon_type:
                pokemon_by_type[pokemon_type].append(pokemon["name"])

genders = ["male", "female"]
for pokemon_type in pokemon_types:
    for gender in genders:
        filtered_users_df = users_df[users_df["pokemon"].isin(pokemon_by_type[pokemon_type])]
        filtered_users_df = filtered_users_df[filtered_users_df["gender"] == "male"]
        filtered_users_df.to_csv(f"csvs/{pokemon_type}_{gender}_users.csv", index=False)

        try:
            filename = f"{pokemon_type}_{gender}_users.csv"
            file_metadata = {
                "name": filename,
                "parents": [google_drive_folder_id_by_type[pokemon_type]],
            }
            media = MediaFileUpload(f"csvs/{filename}", mimetype="text/plain")
            file = service.files().create(body=file_metadata, media_body=media, fields="id").execute()
            print(
                f"File for type *{pokemon_type}* and gender *{gender}* created successfully with ID: {file.get('id')}"
            )

        except HttpError as err:
            print(f"{err} for type *{pokemon_type}* and gender *{gender}*")

File for type *grass* and gender *male* created successfully with ID: 1TgQ-8BkBNYjjqT1BOCF_RKgGnUlpdmf2
File for type *grass* and gender *female* created successfully with ID: 11BKp2Nky2bovKMqOw7MX1p-gnrDNepm8
File for type *electric* and gender *male* created successfully with ID: 1YxhdoPFA94PzS-symzypZOkfIRw20xHe
File for type *electric* and gender *female* created successfully with ID: 1Y_7Nm61eDLtAbcv71YE3SJD8JMp_I0iW
File for type *flying* and gender *male* created successfully with ID: 1tidDCDZRjrHC2r50lnF0Xth7ahqIcb8T
File for type *flying* and gender *female* created successfully with ID: 16O2WXs6aIG1Mtdzz44n5VHHTUdc1Wl_2
File for type *fire* and gender *male* created successfully with ID: 1IMCVdtu_nzMxGR-TsKVbj9gav1Qskpik
File for type *fire* and gender *female* created successfully with ID: 1ANp0T7Bl7XxEK842NcHgtXCWgcuGPm-6
File for type *water* and gender *male* created successfully with ID: 1M5l45IXuQQmjDrnuF6I4IB9xVgbPRJk-
File for type *water* and gender *female* created 

In [456]:
"""
Tarefa 4

Por fim, crie um Google Sheets com um resumo
"""

credentials_dict = json.load(open(credentials_json))
gc, authorized_user = gspread.oauth_from_dict(credentials_dict)

sh = gc.create("[GREENPEACE] Resumo")

worksheet_1 = sh.add_worksheet(title="Número de pessoas por região do Brasil", rows=100, cols=20)
worksheet_2 = sh.add_worksheet(title="Elemento de pokemons por região do Brasil", rows=100, cols=20)

sh.del_worksheet(sh.worksheet("Página1"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=258343893071-cjl374q7gmric124p10qb9kncgkieqmd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A48685%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=5CAOWX4SUx25hZvMojt4rKl3c4IfUh&access_type=offline


{'spreadsheetId': '1Xq9HjSUPDxW0aO6AO-B1cdTxyCK5iPDrfReOh8Gkg9w',
 'replies': [{}]}

In [495]:
"""
Tarefa 4.a

Indique o número de pessoas por região do Brasil (Norte, Sul, Sudeste, Nordeste, Centro-Oeste)
"""

regions = {
    "Norte": [
        "Amazonas",
        "Acre",
        "Tocantins",
        "Rondônia",
        "Pará",
        "Roraima",
        "Amapá",
    ],
    "Sul": [
        "Rio Grande do Sul",
        "Santa Catarina",
        "Paraná",
        "Rio Grande do Sul",
    ],
    "Sudeste": [
        "São Paulo",
        "Rio de Janeiro",
        "Minas Gerais",
        "Espírito Santo",
    ],
    "Nordeste": [
        "Pernambuco",
        "Bahia",
        "Ceará",
        "Rio Grande do Norte",
        "Paraíba",
        "Maranhão",
        "Piauí",
        "Sergipe",
        "Alagoas",
    ],
    "Centro-Oeste": [
        "Goiás",
        "Mato Grosso",
        "Mato Grosso do Sul",
        "Distrito Federal",
    ],
}


def get_region(state):
    for region, states in regions.items():
        if state in states:
            return region


def get_pokemon_types(pokemon):
    pokemon_types = []
    for pokemon_type, pokemon_list in pokemon_by_type.items():
        if pokemon in pokemon_list:
            pokemon_types.append(pokemon_type)
    return pokemon_types


extended_users_df = users_df.copy()
extended_users_df["region"] = users_df["state"].apply(get_region)
users_by_region_df = extended_users_df["region"].value_counts().to_frame()
print(users_by_region_df.to_markdown())

users_by_region_df.reset_index(inplace=True)
users_by_region_df.columns = ["Região", "Número de pessoas"]
worksheet_1.update([users_by_region_df.columns.values.tolist()] + users_by_region_df.values.tolist())


| region       |   count |
|:-------------|--------:|
| Sudeste      |     203 |
| Nordeste     |     151 |
| Sul          |     143 |
| Norte        |     103 |
| Centro-Oeste |      68 |


{'spreadsheetId': '1Xq9HjSUPDxW0aO6AO-B1cdTxyCK5iPDrfReOh8Gkg9w',
 'updatedRange': "'Número de pessoas por região do Brasil'!A1:B6",
 'updatedRows': 6,
 'updatedColumns': 2,
 'updatedCells': 12}

In [496]:
"""
Tarefa 4.b

Indique o número de elementos de Pokemon por região do Brasil
"""


types_by_region = {}
for region in regions.keys():
    types_by_region[region] = {}
    for pokemon_type in pokemon_types:
        types_by_region[region][pokemon_type] = 0

for index, row in users_df.iterrows():
    for pokemon_type in get_pokemon_types(row.pokemon):
        types_by_region[get_region(row.state)][pokemon_type] += 1

types_by_region_df = pd.DataFrame(types_by_region)
print(types_by_region_df.head().to_markdown())

types_by_region_df.index.name = "Elemento"
types_by_region_df.reset_index(inplace=True)
worksheet_2.update([types_by_region_df.columns.values.tolist()] + types_by_region_df.values.tolist())


|          |   Norte |   Sul |   Sudeste |   Nordeste |   Centro-Oeste |
|:---------|--------:|------:|----------:|-----------:|---------------:|
| grass    |      35 |    46 |        64 |         49 |             19 |
| electric |       2 |    10 |         6 |          4 |              0 |
| flying   |       5 |     4 |         5 |          4 |              7 |
| fire     |      27 |    35 |        64 |         55 |             19 |
| water    |      39 |    52 |        69 |         43 |             30 |


{'spreadsheetId': '1Xq9HjSUPDxW0aO6AO-B1cdTxyCK5iPDrfReOh8Gkg9w',
 'updatedRange': "'Elemento de pokemons por região do Brasil'!A1:F7",
 'updatedRows': 7,
 'updatedColumns': 6,
 'updatedCells': 42}